## Amazon API Gateway, AWS Lambda and Amazon S3

This is a basic CDK TypeScript example that deploys 2 AWS Lambda functions. One function will be used to put an object into Amazon S3, the second Lambda function is used to generate a pre-signed url that will be used to download the object from Amazon S3.

![architecture](../images/architecture_2.png "Architecture")


1. The first script will send a request to the Amazon API Gateway, the AWS Lambda integrated with the API will write the request body as an object to Amazon S3.


2. The second command requests a pre-signed URL through the Amazon API Gateway.


**Note:** Make sure you set gwid to your gateway id using - gwid = '...'

**Post order**


In [ ]:
import json, boto3, requests, datetime

#Set gateway id
gwid = '...'
    
url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/invoice')
    
for i in range(5):
    x = datetime.datetime.now()
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(x)
    coffeetype = 'Short Black'
    coffeesize = 'Small'
    unitprice = str(4.50 * (i+1))
    quantity = str(i+1)

    response = requests.post(url,json={'order':{
                'accountid': accountid,
                'vendorid': vendorid,
                'orderdate':orderdate,
                'details':{
                    'coffeetype': coffeetype,
                    'coffeesize': coffeesize,
                    'unitprice': unitprice,
                    'quantity': quantity
                }
            }
        })
    print(response)

**json formatter** - Run the following script to create a class which will be used to render json objects in a readable format.

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

**Get pre-signed url from S3**, use the url to retrieve the object.

In [ ]:
#Get Script

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/invoice')

now = datetime.datetime.now()
path = str(now.year) + '/' + str(now.month) + '/'

response_get = requests.get(url, params={'objectKey':path + 'a0'})

print(response_get.json())